# <font color= 'purple'>Andrielly Pereira </font>

## <font color ='black'>Desafio FADESP Processo Seletivo</font>

In [ ]:
# Versão linguagem Python
from platform import python_version
print('Versão de Linguagem Python Usado Neste Jupyter Notebook:', python_version())

## Inicializando o Projeto

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

### Carregando os Pacotes

In [ ]:
# manipulação e Visualização de dados
import time
import pickle
import sklearn
import datetime
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib as m
import matplotlib.pyplot as plt


# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
sns.set_style('whitegrid')
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Formatação dos Gráficos
plt.style.use('fivethirtyeight')
plt.figure(1, figsize = (15, 6))
%matplotlib inline

In [ ]:
# Versões dos Pacotes usados
%reload_ext watermark
%watermark -a "Andrielly Moraes" --iversions

## Carregando os Dados

In [ ]:
# Carregue o dataset
filename = 'https://raw.githubusercontent.com/andriellymoraespereira/desafio-fadesp-processo-seletivo/master/data/dataset_desafio_fadesp.csv'
df = pd.read_csv(filename, index_col = 0, encoding = 'latin-1')

Armagenando o dataset em um banco de dados chave-valor SQLite 

In [ ]:
conn = sqlite3.connect('enterprise.db')

In [ ]:
df.to_sql(name='sales', con=conn)

In [ ]:
!sqlite3 enterprise.db ".tables"

In [ ]:
# salvando os dados
!sqlite3 enterprise.db "PRAGMA table_info(sales)"

In [ ]:
# recuperando os dados direto em um pandas dataframe
df_read = pd.read_sql('SELECT * FROM sales', con = conn)

In [ ]:
type(df_read)

In [ ]:
df_read

In [ ]:
query = """
           SELECT *
           FROM sales;
        """
df_read = pd.read_sql(query, con=conn, index_col="Row ID")
df_read 

Criando uma nova variavel para a quantidade total vendida.

In [ ]:
query = """
           SELECT *, "Quantity" * "Sales" AS "total"
           FROM sales;
        """
df_read = pd.read_sql(query, con=conn, index_col="Row ID")
df_read 

Salvando os novos dados na tabela sales

In [ ]:
df_read.to_sql(name='sales', con=conn, if_exists = 'replace')

In [ ]:
df_read = pd.read_sql('SELECT * FROM sales', con=conn)
df_read

In [ ]:
# fechando a conexão com o banco de dados
conn.close()

### Análise Exploratória

Vamos explorar os dados por diferentes perspectivas e compreender um pouco mais o relacionamento entre as variáveis.

In [ ]:
# Colunas
df.columns

In [ ]:
# Renomeado as colunas
df = df.rename(columns={
    "Row ID": "Row_ID",
    "Order ID": "Order_ID",
    "Order Date": "Order_Date",
    "Ship Date": "Ship_Date",
    "Ship Mode": "Ship_Mode",
    "Customer ID": "Customer_ID",
    "Customer Name": "Customer_Name",
    "Product ID": "Product_ID",
    "Sub-Category": "Sub_Category",
    "Product Name": "Product_Name",
    "Shipping Cost": "Shipping_Cost",
    "Order Priority": "Order_Priority"
})

In [ ]:
# Verificação de total de valores únicos por coluna
df.nunique()

In [ ]:
# Resumo das colunas numéricas
df.describe()

In [ ]:
# Verificando dados nulos
df.info()

Para esta análise vamos excluir a variavel Postal Code 

In [ ]:
df = df.drop(columns=['Postal Code'])

In [ ]:
# Convertendo a coluna de data para o tipo data
df.Order_Date  = pd.to_datetime(df.Order_Date)
df.Ship_Date  = pd.to_datetime(df.Ship_Date)

In [ ]:
# tipos de dados
df.dtypes

In [ ]:
# Range de datas do periodo que ocorreram as vendas
print('Data Mínima:', df['Order_Date'].min())
print('Data Maxíma:', df['Order_Date'].max())

Agora vamos conhecer os valores dentro das váriaveis region, category, Ship Mode, Market

In [ ]:
# Contagem (Region)
df['Region'].value_counts()

In [ ]:
# Contagem (Category)
df['Category'].value_counts()

In [ ]:
# Contagem (Ship Mode)
df['Ship_Mode'].value_counts()

In [ ]:
# Contagem (Market)
df['Market'].value_counts()

Identificão dos produtos mais vendidos e analisando sua distribuição geográfica.

In [ ]:
df.groupby(['Country','Product_Name'])['Quantity'].sum().sort_values(ascending=False).reset_index()

O produto mais vendido foi Staples e, seu maior produtor é United States.

In [ ]:
# BoxPlot do Valor da venda realizada por Modo de envio do pedido.
ax = df.plot.box(column="Sales", by="Ship_Mode", figsize=(10, 8))

O modo de envio classe padrão apresdentar os maiores outlirs em relação ao Valor da venda realizada.

In [ ]:
# Correlação (tabela)
df.corr()

In [ ]:
# Correlação (visual)
sns.pairplot(df)

In [ ]:
# análise da relação entre Disconto(Discount) e valor da venda realisada (sales)
# Plot
plt.figure(figsize = (10, 8))
sns.set(font_scale = 1.1)
sns.jointplot(data = df, 
              x = 'Sales', 
              y = 'Discount',
              color = 'blue')

Observando os histogramas, podemos ver que os dados não tem distribuídos normal.

Observando o gráfico de dispersão, não parece haver correlação entre as duas variáveis. Vamos confirmar calculando o coeficiente de correlação entre elas.

In [ ]:
# Correlação
df[['Sales', 'Discount']].corr()

Há uma correlação inversa entre o Desconto realizado e  Valor da venda realizada, os descontos impulsionam as vendas.

In [ ]:
# análise da relação entre Quantidade de produtos vendidos e o Valor do frete.
# Plot
plt.figure(figsize = (10, 8))
sns.set(font_scale = 1.1)
sns.jointplot(data = df, 
              x = 'Sales', 
              y = 'Shipping_Cost',
              color = 'blue')

Observando os histogramas, podemos ver que os dados não tem distribuídos normal.

Observando o gráfico de dispersão, não parece haver correlação entre as duas variáveis. Vamos confirmar calculando o coeficiente de correlação entre elas.

In [ ]:
# Correlação
df[['Sales', 'Shipping_Cost']].corr()

Há uma alta correlação positiva entre a Quantidade de produtos vendidos e o Valor do frete.

Agora vamos separamos os dados em dois conjuntos (2013 e 2014) para exibir a evolução do lucro bruto ao longo de cada ano. Plotando no mesmo gráfico podemos fazer um comparativo dos anos mês a mês.

In [ ]:
#Selecionando apenas as vendas de 2013
df_2013 = df[df["Order_Date"].dt.year == 2013]
#Selecionando apenas as vendas de 2014
df_2014 = df[df["Order_Date"].dt.year == 2014]
# Comparativo de lucro bruto por mês
df_2013.groupby(df_2013["Order_Date"].dt.month)["Profit"].sum().plot(color='red', label='Ano 2013')
df_2014.groupby(df_2014["Order_Date"].dt.month)["Profit"].sum().plot(color='green', label='Ano 2014')
plt.title("Lucro x Mês")
plt.xlabel("Mês")
plt.ylabel("Lucro Bruto")
plt.legend()
plt.show()

No gráfico Lucro x Mês, verifica-se que o ano de 2014 apresentou maior lucro por mês, comparado ao ano de 2013.

## Pré-Processamento de Dados Para Construção de Modelos de Machine Learning.

In [ ]:
# Variáveis de entrada (variáveis preditoras)
X = df[['Quantity', 
        'Discount',
        'Profit', 
        'Shipping_Cost']]

In [ ]:
# Variável de saída (variável alvo ou target)
y = df['Sales']

In [ ]:
# Divisão em dados de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.3, random_state = 101)

In [ ]:
len(X_treino)

In [ ]:
len(X_teste)

### Padronização

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X_treino)

In [ ]:
X_treino = scaler.transform(X_treino)

In [ ]:
X_teste = scaler.transform(X_teste)

In [ ]:
X_treino[:5]

In [ ]:
X_teste[:5]

# Construção, Treinamento e Avaliação do Modelo de Regressão Linear (Benchmark)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# Modelo
modelo = LinearRegression()

In [ ]:
# Treinamento
modelo.fit(X_treino, y_treino)

In [ ]:
print('Coeficientes: \n', modelo.coef_)

In [ ]:
# Coeficientes das variáveis preditoras
df_coef = pd.DataFrame(modelo.coef_, X.columns, columns = ['Coeficiente'])
df_coef

### Avaliação do Modelo

In [ ]:
# Previsões com dados de teste
pred = modelo.predict(X_teste)

In [ ]:
# Imprime as 10 primeiras previsões
pred[:10]

In [ ]:
plt.figure(figsize = (10,8))
plt.scatter(x = y_teste, y = pred, color = 'green', edgecolors = 'blue')
plt.xlabel('Valor Real de Y')
plt.ylabel('Valor Previsto de Y');

A partir do gráfico de dispersão, podemos ver que há uma correlação psitiva entre os y's previstos e os y's reais nos dados do teste.

In [ ]:
# Valor médiode venda realizada pelos clientes
df['Sales'].mean()

In [ ]:
# Valor mínimo
df['Sales'].min()

In [ ]:
# Valor máximo
df['Sales'].max()

In [ ]:
# MAE - Erro Médio Absoluto
mean_absolute_error(y_teste, pred)

O MAE prevê que, em média, as previsões do nosso modelo (Valor da venda realizada) estão erradas em aproximadamente 117.57 reais, o que é aproximadamente a metade do valor comparado ao valor médio de venda realizada por cliente.

In [ ]:
# MSE - Erro quadrático médio
mean_squared_error(y_teste, pred)

In [ ]:
# MAPE
MAPE = np.mean(np.abs((y_teste - pred) / y_teste)) * 100
print(f"Valor do MAPE dado em percentual: {MAPE}")

In [ ]:
# Coeficiente R2
r2_score(y_teste, pred)

In [ ]:
# Variância Explicada
explained_variance_score(y_teste, pred)

Nosso modelo é capaz de explicar 70% da variância dos dados, o que é bom.

O coeficiente R² = 0,.6957 significa que o modelo linear explica 69,57% da variância da variável dependente a partir do regressores (variáveis independentes) incluídas no modelo linear.

### Resíduos

In [ ]:
# Plot
plt.figure(figsize = (8,4))
ax = sns.distplot((y_teste - pred), 
                  bins = 40, 
                  color = 'red', 
                  hist_kws = dict(edgecolor = 'black', linewidth = 0.3))
ax.set(xlim = (-40, 40))
ax.set(ylim = (0, 0.055));

In [ ]:
# arquivo Pickle 
serialized = pickle.dumps(modelo)
  
print(serialized)